In [1]:
!pip uninstall -y ale-py gymnasium

Found existing installation: ale-py 0.11.2
Uninstalling ale-py-0.11.2:
  Successfully uninstalled ale-py-0.11.2
Found existing installation: gymnasium 1.2.2
Uninstalling gymnasium-1.2.2:
  Successfully uninstalled gymnasium-1.2.2


In [2]:
!pip install stable-baselines3[extra]
!pip install "autorom[accept-rom-license]"


  Using cached gymnasium-1.2.2-py3-none-any.whl.metadata (10 kB)
  Using cached ale_py-0.11.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (9.0 kB)
Using cached gymnasium-1.2.2-py3-none-any.whl (952 kB)
Using cached ale_py-0.11.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (5.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ale-py]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires jax>=0.1.72, which is not installed.
dopamine-rl 4.1.2 requires jaxlib>=0.1.51, which is not installed.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.


In [3]:
!pip install gymnasium>=1.0.0 ale-py>=0.9.0 shimmy>=1.0.0


In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')

import ale_py
import gymnasium as gym
gym.register_envs(ale_py)
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd
import numpy as np
import random
import torch
import gc

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [5]:
gc.collect()
torch.cuda.empty_cache()

In [6]:
CONFIG_ID = 5

configs = [
    {"lr": 0.00035, "gamma": 0.99,  "batch": 12, "eps_start": 1.0, "eps_end": 0.03, "eps_decay": 0.25},
    {"lr": 0.00075, "gamma": 0.97,  "batch": 20, "eps_start": 1.0, "eps_end": 0.07, "eps_decay": 0.15},
    {"lr": 0.00008, "gamma": 0.992, "batch": 10, "eps_start": 1.0, "eps_end": 0.02, "eps_decay": 0.3},
    {"lr": 0.00045, "gamma": 0.985, "batch": 18, "eps_start": 1.0, "eps_end": 0.04, "eps_decay": 0.2},
    {"lr": 0.00028, "gamma": 0.98,  "batch": 14, "eps_start": 1.0, "eps_end": 0.015, "eps_decay": 0.18},
    {"lr": 0.00055, "gamma": 0.99,  "batch": 10, "eps_start": 1.0, "eps_end": 0.05, "eps_decay": 0.22},
    {"lr": 0.0009,  "gamma": 0.975, "batch": 12, "eps_start": 1.0, "eps_end": 0.08, "eps_decay": 0.12},
    {"lr": 0.00012, "gamma": 0.996, "batch": 16, "eps_start": 1.0, "eps_end": 0.01, "eps_decay": 0.28},
    {"lr": 0.0004,  "gamma": 0.988, "batch": 15, "eps_start": 1.0, "eps_end": 0.035, "eps_decay": 0.19},
    {"lr": 0.00018, "gamma": 0.993, "batch": 13, "eps_start": 1.0, "eps_end": 0.02, "eps_decay": 0.16},
]


cfg = configs[CONFIG_ID - 1]


print(f"Policy: CnnPolicy")
print(f"Hyperparameters:")
print(f"  • Learning Rate (lr): {cfg['lr']}")
print(f"  • Gamma (γ): {cfg['gamma']}")
print(f"  • Batch Size: {cfg['batch']}")
print(f"  • Epsilon Start: {cfg['eps_start']}")
print(f"  • Epsilon End: {cfg['eps_end']}")
print(f"  • Epsilon Decay: {cfg['eps_decay']}")

Policy: CnnPolicy
Hyperparameters:
  • Learning Rate (lr): 0.00028
  • Gamma (γ): 0.98
  • Batch Size: 14
  • Epsilon Start: 1.0
  • Epsilon End: 0.015
  • Epsilon Decay: 0.18


In [7]:
env_test = gym.make("ALE/SpaceInvaders-v5")
random_scores = []

for episode in range(1, 21):
    obs, _ = env_test.reset()
    done = False
    truncated = False
    score = 0
    while not (done or truncated):
        action = random.choice([0, 1, 2, 3, 4, 5])
        obs, reward, done, truncated, _ = env_test.step(action)
        score += reward
    random_scores.append(score)
    if episode % 5 == 0:
        print(f"  Episode {episode}: {score}")

avg_random_score = np.mean(random_scores)
print(f"\nRandom Agent Average Score: {avg_random_score:.2f}")
env_test.close()

  Episode 5: 125.0
  Episode 10: 120.0
  Episode 15: 165.0
  Episode 20: 135.0

Random Agent Average Score: 157.25


In [8]:
env = gym.make("ALE/SpaceInvaders-v5", frameskip=4)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4)

In [9]:
model = DQN(
    "CnnPolicy",
    env,
    learning_rate=cfg["lr"],
    gamma=cfg["gamma"],
    batch_size=cfg["batch"],
    buffer_size=5000,
    learning_starts=2000,
    exploration_initial_eps=cfg["eps_start"],
    exploration_final_eps=cfg["eps_end"],
    exploration_fraction=cfg["eps_decay"],
    target_update_interval=1000,
    train_freq=4,
    gradient_steps=1,
    tensorboard_log=f"./logs/",
    verbose=1,
    device="cuda"
)

print("Model created successfully!")
print(f"   Buffer size: {model.buffer_size:,}")
print(f"   Learning starts at: {model.learning_starts:,} steps")
print(f"   Policy: CnnPolicy (Convolutional Neural Network)")

print(f"\nInitial GPU memory: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

Using cpu device
Wrapping the env in a VecTransposeImage.
Model created successfully!
   Buffer size: 5,000
   Learning starts at: 2,000 steps
   Policy: CnnPolicy (Convolutional Neural Network)

Initial GPU memory: 0.00 GB


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [10]:
try:
    model.learn(
        total_timesteps=100_000,
        tb_log_name=f"SpaceInvaders_Config{CONFIG_ID}_CnnPolicy",
        log_interval=1000,
        progress_bar=True
    )

    print("\nTraining completed successfully!")
    print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")

except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print(f"\nOUT OF MEMORY ERROR")
        print(f"Current settings: buffer={model.buffer_size}, batch={cfg['batch']}")
        print(f"\nEMERGENCY FIX:")
        print(f"   1. Change buffer_size=5000")
        print(f"   2. Change batch_size=4")
        print(f"   3. Or use device='cpu' (slower but stable)")
    raise e

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 99,997/100,000  [ 4:28:41 < 0:00:01 , 6 it/s ]


Training completed successfully!
Max GPU memory used: 0.00 GB


In [11]:
mean_reward, std_reward = evaluate_policy(
    model,
    env,
    n_eval_episodes=10,
    deterministic=True
)

episode_lengths = []

for episode in range(1, 11):
    obs = env.reset()
    done = False
    length = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        length += 1

    episode_lengths.append(length)
    print(f"  Episode {episode}: {length} steps")

total_episode_length = sum(episode_lengths)

print(f"\nResults over 10 episodes:")
print(f"   Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"   Random Baseline: {avg_random_score:.2f}")
print(f"   Improvement: {mean_reward - avg_random_score:.2f}")
print(f" total episode lenght: {total_episode_length}")

  Episode 1: 749 steps
  Episode 2: 749 steps
  Episode 3: 749 steps
  Episode 4: 749 steps
  Episode 5: 749 steps
  Episode 6: 749 steps
  Episode 7: 619 steps
  Episode 8: 749 steps
  Episode 9: 746 steps
  Episode 10: 749 steps

Results over 10 episodes:
   Mean Reward: 421.00 ± 78.03
   Random Baseline: 157.25
   Improvement: 263.75
 total episode lenght: 7357


In [ ]:
model_name = "dqn_model_5"
model.save(model_name)

backup_name = f"dqn_spaceinvaders_exp{CONFIG_ID}"
model.save(backup_name)

print(f"\nModels saved:")
print(f"   • {model_name}.zip (required submission)")
print(f"   • {backup_name}.zip (backup with config ID)")